In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.python import keras
import pandas as pd
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.python.keras import utils
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from tensorflow.python import RandomFlip, RandomRotation, Rescaling

AttributeError: module 'keras.engine.base_layer' has no attribute 'BaseRandomLayer'

In [ ]:
# import tensorflow as tf
import numpy as np
# from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
batch_size = 32
image_size = (130, 130)

train_dataset = image_dataset_from_directory('C:\dataset\m_learning\cell_images\cell_images_1',
                                             subset = 'training',
                                             seed = 69,
                                             validation_split = 0.1,
                                             batch_size = batch_size,
                                             image_size = image_size)

In [ ]:
validation_dataset = image_dataset_from_directory('C:\dataset\m_learning\cell_images\cell_images_1',
                                                  subset='validation',
                                                  seed=69,
                                                  validation_split=0.1,
                                                  batch_size=batch_size,
                                                  image_size=image_size)

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 2)
val_data = validation_dataset.skip(val_batches // 2)

In [ ]:
class_names = train_dataset.class_names
class_names

In [ ]:
plt.figure(figsize=(12, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"), cmap="gray")
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_data = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_data = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential(
    [
        RandomFlip('horizontal'),
        RandomRotation(0.25),
    ], "Augmentation_Layers")

In [ ]:
for image, _ in train_dataset.take(1):
    plt.figure(figsize=(10, 10))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0] / 255, cmap="gray")
        plt.axis('off')

In [ ]:
# Создаем последовательную модель
model = Sequential()
# Сверточный слой
model.add(Conv2D(16, (5, 5), padding='same',
                 input_shape=(130, 130, 3), activation='relu'))
# Слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Сверточный слой
model.add(Conv2D(32, (5, 5), activation='relu', padding='same'))
# Слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Сверточный слой
model.add(Conv2D(32, (5, 5), activation='relu', padding='same'))
# Слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Сверточный слой
model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
# Слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Полносвязная часть нейронной сети для классификации
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
# Выходной слой, 131 нейрон по количеству классов
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])


In [ ]:
history = model.fit(train_dataset,
                    validation_data=validation_dataset,
                    epochs=5)
                    # verbose=2)

In [ ]:
# Оцениваем качество обучения модели на тестовых данных
scores = model.evaluate(test_dataset, verbose=1)

In [ ]:
plt.plot(history.history['accuracy'],
         label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_accuracy'],
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

In [ ]:
d1 = pd.DataFrame(history.history)
d1.plot()